데이터베이스 내의 최신 값과 비교 후 없는 데이터만 파일로 서버에 저장하는 방식입니다.

In [1]:
## K-STAT 국가별 데이터 크롤링 - 수출데이터
from selenium import webdriver
import time
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import os, warnings
import pandas as pd
import glob
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import pymysql
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from os import path
import shutil

# MySQL Connector using pymysql
pymysql.install_as_MySQLdb()
import MySQLdb

# 개발 서버
conn= MySQLdb.connect(user='root', passwd='', host='', db='')
cur=conn.cursor() 

op = Options()
op.add_experimental_option('prefs',{'download.default_directory':r'C:\Users\jeongyewon\Documents\kstat_test'})

driver = webdriver.Chrome('C:/chrome/chromedriver', options=op)

C:\Users\JEONGY~2\AppData\Local\Temp/ipykernel_29744/3249782889.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/chrome/chromedriver', options=op)


In [2]:
cur.execute("SELECT SUBSTR(MAX(BASE_DATE),1,4) FROM KSTAT_ITEM_EXPORT;")
max_year = cur.fetchone()

cur.execute("SELECT MAX(BASE_DATE) FROM KSTAT_ITEM_EXPORT;")
max_date = cur.fetchone()

In [ ]:
# db에서 현재 보유한 date max 값을 조회 
# 현재 날짜 조회
# date max 값과 현재 날짜 사이의 값들을 크롤링 진행
# 크롤링 후 파일을 저장하고, 저장된 파일의 경로를 이동한다.

# 이동된 경로의 파일을 db 적재 (이는 ftp 업로드 방식과 동일하게 진행)

In [3]:
# k-stat 한국무역협회 접속
driver.get('https://stat.kita.net/main.screen')

In [4]:
#로그인 선택
driver.find_element_by_xpath('//*[@id="header"]/div/div[2]/ul/li[2]/a').click()
driver.implicitly_wait(1)

C:\Users\JEONGY~2\AppData\Local\Temp/ipykernel_29744/1245614441.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="header"]/div/div[2]/ul/li[2]/a').click()


In [5]:
# id, pw 작성
driver.find_element_by_xpath('//*[@id="userId"]').send_keys("")
driver.find_element_by_xpath('//*[@id="pwd"]').send_keys("")
driver.find_element_by_xpath('//*[@id="loginBtn"]').click()
# 로그인 되는 동안 다른 작업시 문제 발생, 10초간 대기
time.sleep(10)

C:\Users\JEONGY~2\AppData\Local\Temp/ipykernel_29744/1976247991.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="userId"]').send_keys("ulsan02")
C:\Users\JEONGY~2\AppData\Local\Temp/ipykernel_29744/1976247991.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="pwd"]').send_keys("ckdwh6156")
C:\Users\JEONGY~2\AppData\Local\Temp/ipykernel_29744/1976247991.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="loginBtn"]').click()


In [6]:
# 국내통계 지자체 수출입 접속 / 클릭이 불가능해서 '지자체 수출입'-'품목별' 웹사이트 바로 접속
driver.get("https://stat.kita.net/stat/kts/prod/ProdItemImpExpList.screen")
driver.implicitly_wait(1)

driver.find_element_by_xpath('//*[@id="s_prod_code"]').click()
driver.find_element_by_xpath('//*[@id="s_prod_code"]').clear()

C:\Users\JEONGY~2\AppData\Local\Temp/ipykernel_29744/3929082813.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="s_prod_code"]').click()
C:\Users\JEONGY~2\AppData\Local\Temp/ipykernel_29744/3929082813.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="s_prod_code"]').clear()


In [7]:
htmlsrc = driver.page_source

bs = BeautifulSoup(htmlsrc, "html.parser")

# <select> 태그 중 class 이름이 "s_year","s_month"인 것 전부 찾기
years = bs.find("select", {"name" : "s_year"})
months = bs.find("select", {"name" : "s_month"})

In [8]:
years = years.find_all('option')
months = months.find_all('option')

In [9]:
latest_date = years[0]['value']+months[len(months)-1]['value']

In [10]:
# us_list : 다운받을 시도, 시군구 코드
# year_list = 다운받을 년도
# month_list = 다운받을 월

us_list = [68]
next_year = datetime.now()+timedelta(weeks=52)
year_list = range(int(max_year[0]),int(next_year.strftime('%Y')))
month_list = range(1,13)

if int(latest_date) > max_date[0] :

    #품목 단위 선택 - 'MTI'
    Select(driver.find_element_by_xpath('//*[@id="s_cond_gb"]')).select_by_visible_text(str('MTI'))
    time.sleep(1)

    #품목 단위수 선택 - '3단위'
    Select(driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[1]/div/select[2]')).select_by_visible_text(str('3단위'))
    time.sleep(1)

    # 당월/누계 클릭 (당월)
    Select(driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[4]/select')).select_by_visible_text('당월')
    time.sleep(1)

    # 단위 클릭 (백만불, %)
    Select(driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[5]/select')).select_by_visible_text('백만불, %')
    time.sleep(1)

    # 정렬기준 클릭 (수출금액)
    Select(driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[6]/select')).select_by_visible_text('수출금액')
    time.sleep(1)


    for us_code in us_list:
        # 시도 입력 초기화
        driver.find_element_by_xpath('//*[@id="s_prod_code"]').clear()
        # 시도 입력
        driver.find_element_by_xpath('//*[@id="s_prod_code"]').send_keys(us_code)
        
        for year in year_list:
            # select year클릭
            Select(driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[1]/select')).select_by_visible_text(str(year))
            time.sleep(1)
            
            tmp = BeautifulSoup(htmlsrc, "html.parser")
            tmp_months = tmp.find("select", {"name" : "s_month"})
            tmp_months = tmp_months.find_all('option')
            
            month_list = range(1,int(months[len(months)-1]['value'])+1)
            
            for month in month_list:
                # select 월 클릭
                Select(driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[2]/select')).select_by_visible_text(str(month))
                time.sleep(1)
                
                # 수출금액 클릭
                Select(driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[6]/select')).select_by_visible_text('수출금액')
                time.sleep(1)

                # 조회 클릭
                driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[3]/a').click()
                time.sleep(3)

                # 엑셀 다운로드 클릭
                driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/div[1]/div/a[1]').click()
else : 
    print('dont need to update')

C:\Users\JEONGY~2\AppData\Local\Temp/ipykernel_29744/1122943755.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Select(driver.find_element_by_xpath('//*[@id="s_cond_gb"]')).select_by_visible_text(str('MTI'))
C:\Users\JEONGY~2\AppData\Local\Temp/ipykernel_29744/1122943755.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Select(driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[1]/div/select[2]')).select_by_visible_text(str('3단위'))
C:\Users\JEONGY~2\AppData\Local\Temp/ipykernel_29744/1122943755.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Select(driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[4]/select')).select_by_visible_text('당월')
C:\Users\JEONGY~2\AppData\Local\Temp/ipykernel_29744/1122943755.py:25: DeprecationWarning: find_element_by_* commands are depr

In [11]:
# 서버 내 경로 설정에 따라 값 수정필요

file_path_dir = 'C:/Users/jeongyewon/Documents/kstat_test/'
fin_path_dir = 'C:/Users/jeongyewon/Documents/kstat_test_fin/'

kstat_path_dir = 'C:/Users/jeongyewon/Documents/kstat_test_fin/'

In [12]:
if path.exists(file_path_dir) : 
    tot_data = pd.DataFrame()

    for file in glob.glob(file_path_dir+'*.xls'): 
        df = pd.read_excel(file)
        a = df.iat[0,0]
        b = a[34:38]
        c = a[40:42]
        df = df.drop(index=[0,1])
        df = df.rename(columns=df.iloc[0])
        df.rename(columns={'순번':'BASE_DATE'}, inplace = True)
        df = df.drop(index=[2])
        df['BASE_DATE'] = b + c
        tot_data = tot_data.append(df, ignore_index=True)
        
    tot_data.columns = ['BASE_DATE','ITEM_CD_MTI','ITEM_NM_MTI','1','2','3','4','5','EX_AMT','EX_AMT_RATE','IM_AMT','IM_AMT_RATE','PROFIT_AMT']
    tot_data['REGION_NM'] = 'US'
    tot_data = tot_data[['BASE_DATE','REGION_NM','ITEM_CD_MTI','ITEM_NM_MTI','EX_AMT','EX_AMT_RATE','IM_AMT','IM_AMT_RATE','PROFIT_AMT']]
    
    tot_data['BASE_YEAR'] = tot_data['BASE_DATE'].str[0:4]
    tot_data['BASE_MONTH'] = tot_data['BASE_DATE'].str[4:]
    
    tot_data['BASE_MONTH'] = np.where(tot_data['BASE_MONTH'] == '1월','01',tot_data['BASE_MONTH'])
    tot_data['BASE_MONTH'] = np.where(tot_data['BASE_MONTH'] == '2월','02',tot_data['BASE_MONTH'])
    tot_data['BASE_MONTH'] = np.where(tot_data['BASE_MONTH'] == '3월','03',tot_data['BASE_MONTH'])
    tot_data['BASE_MONTH'] = np.where(tot_data['BASE_MONTH'] == '4월','04',tot_data['BASE_MONTH'])
    tot_data['BASE_MONTH'] = np.where(tot_data['BASE_MONTH'] == '5월','05',tot_data['BASE_MONTH'])
    tot_data['BASE_MONTH'] = np.where(tot_data['BASE_MONTH'] == '6월','06',tot_data['BASE_MONTH'])
    tot_data['BASE_MONTH'] = np.where(tot_data['BASE_MONTH'] == '7월','07',tot_data['BASE_MONTH'])
    tot_data['BASE_MONTH'] = np.where(tot_data['BASE_MONTH'] == '8월','08',tot_data['BASE_MONTH'])
    tot_data['BASE_MONTH'] = np.where(tot_data['BASE_MONTH'] == '9월','09',tot_data['BASE_MONTH'])
    
    tot_data['BASE_DATE'] = tot_data['BASE_YEAR'].map(str)+tot_data['BASE_MONTH']
    tot_data['BASE_DATE'] = tot_data['BASE_DATE'].astype(int)
    tot_data = tot_data[tot_data.BASE_DATE > max_date[0]]
    
    tot_data = tot_data.drop(['BASE_YEAR','BASE_MONTH'], axis='columns')
    
    tot_data = tot_data.sort_values(by=['BASE_DATE'], ascending=[True])
    tot_data = tot_data.reset_index(drop=True)
    
    tot_data.to_csv(kstat_path_dir+'KSTAT_ITEM_EXPORT.csv', index=False, encoding='euc-kr')

In [13]:
file_list = os.listdir(file_path_dir)
file_list_xls = [file for file in file_list if file.endswith('.xls')]

for i in range(len(file_list_xls)):
    shutil.move(file_path_dir+file_list_xls[i], fin_path_dir+file_list_xls[i])

In [14]:
conn.close()